In [1]:
import numpy as np
import pandas as pd

from src.M4_experiment import M4_parser
from src.utils.data import long_to_wide
from src.ESRNN import ESRNN

In [2]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')


def plot_prediction(y, y_hat, forecast=True):
    """
    """
    n_y = len(y)
    n_yhat = len(y_hat)
    ds_y = np.array(range(n_y))
    ds_yhat = np.array(range(n_y, n_y+n_yhat))

    plt.plot(ds_y, y, label = 'y')
    plt.plot(ds_yhat, y_hat, label='y_hat')
    plt.legend(loc='upper left')
    plt.show()

In [3]:
dataset = M4_parser(dataset_name='Quarterly', mode='train', num_obs=100)
X, y = long_to_wide(dataset)
print(X[:5])
print(y[:5])

[['Q1' 'Macro']
 ['Q10' 'Macro']
 ['Q100' 'Macro']
 ['Q11' 'Macro']
 ['Q12' 'Macro']]
[[7407.41231382 7528.5660743  7374.70922497 ...           nan
            nan           nan]
 [2329.         2349.9        2332.9        ...           nan
            nan           nan]
 [5036.         5089.         5226.         ...           nan
            nan           nan]
 [1082.5        1076.1        1063.1        ...           nan
            nan           nan]
 [3726.         4048.         4704.         ...           nan
            nan           nan]]


In [4]:
esrnn = ESRNN(max_epochs=2, batch_size=2)
esrnn.fit(X,y)
#pred_ini = esrnn.predict()

========== Training ESRNN ==========
========= Epoch 0 finished =========
Training time: 3.179318904876709
Forecast loss: 0.0870470330119133
========= Epoch 1 finished =========
Training time: 3.152277946472168
Forecast loss: 0.07972340285778046
Train finished!


In [ ]:
plot_prediction(X[X['unique_id']==unique_id]['y'], pred_ini[pred_ini['unique_id']==unique_id]['y_hat'])

In [ ]:
esrnn = ESRNN(max_epochs=5)
esrnn.fit(X,y)
preds = esrnn.predict()

In [ ]:
plot_prediction(X[X['unique_id']==unique_id]['y'], preds[preds['unique_id']==unique_id]['y_hat'])

In [ ]:
import pandas as pd
import numpy as np

import torch
from src.ESRNN import ESRNN
from src.utils.data import Iterator, long_to_wide
from src.M4_experiment import M4_parser

data = {'unique_id': ["q1", "q1", "q1", "q1", "q2", "q2", "q2", "q3", "q4", "q4", "q4"], 
        'ts': [1, 2, 3, 4, 1, 2, 3, 1, 1, 2, 3],
        'x': ["a", "a", "a", "a", "b", "b", "b", "c", "d", "d", "d"],
        'y': [1.1, 1.2, 1.3, 1.4, 2.1, 2.2, 2.3, 3.1, 4.1, 4.2, 4.3]}
# dataset = pd.DataFrame.from_dict(data)
dataset = M4_parser('Quarterly', mode='train', num_obs=200, data_dir='./data/m4')

X, y = long_to_wide(dataset)
esrnn = ESRNN()

mc = esrnn.mc
unique_categories = dataset['x'].unique()
mc.category_to_idx = dict((word, index) for index, word in enumerate(unique_categories))
mc.exogenous_size = len(unique_categories)
mc.batch_size = 2

iterator = Iterator(mc=mc, X=X, y=y, shuffle=True)

batch = iterator.get_batch()
print("iterator.sort_key \n", iterator.sort_key)
print("batch.y \n", batch.y)
print("batch.idxs", batch.idxs)
print("batch.last_ts", batch.last_ts)
print("iterator.X \n", iterator.X)